In [1]:
import scanpy as sc
import pandas as pd

/Users/npapadop/mambaforge/envs/ascc23/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import requests
from tqdm import tqdm
def download_file(doi,ext):
    url = 'https://api.datacite.org/dois/'+doi+'/media'
    r = requests.get(url).json()
    netcdf_url = r['data'][0]['attributes']['url']
    r = requests.get(netcdf_url,stream=True)
    #Set file name
    fname = doi.split('/')[-1]+ext
    #Download file with progress bar
    if r.status_code == 403:
        print("File Unavailable")
    if 'content-length' not in r.headers:
        print("Did not get file")
    else:
        with open(fname, 'wb') as f:
            # total_length = int(r.headers.get('content-length'))
            # pbar = tnrange(int(total_length/1024), unit="B")
            for chunk in r.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
        return fname

In [3]:
# download_file('10.22002/D1.1796','.gz')
# download_file('10.22002/D1.1797','.gz')

In [4]:
# !gunzip -f *.gz

In [5]:
raw = sc.read_h5ad("D1.1797")
processed = sc.read_h5ad("D1.1796")

In [6]:
processed.obs["fed"] = processed.obs["fed"].map({"True":True, "False":False})
processed.obs["fed"] = processed.obs["fed"].astype(bool)

In [7]:
output = "/Users/npapadop/Documents/teaching/2023/advanced_scRNAseq/data/clytia/"
processed.obs.to_csv(output + "clytia_metadata.csv")

In [8]:
processed.obs["assignment"] = processed.obs["cellRanger_louvain"].copy()

replace = {31: "Ronit", # neural
           30: "Michelle", # muscle
           21: "Noemi", # nematocysts
           29: "Oliver", # bioluminescence
           35: "Mario", # very early oocytes
           24: "Antonia", # gastrodermis/digestive
}
processed.obs["assignment"].replace(replace, inplace=True)
processed.obs["assignment"] = processed.obs["assignment"].astype(str)

In [9]:
processed.obs = processed.obs[["batch", "orgID", "fed", "starved", "assignment"]]

In [10]:
panther = pd.read_csv("/Users/npapadop/Documents/teaching/2023/advanced_scRNAseq/data/clytia/full_rna_nr_panther.txt",
                      sep="\t", index_col=0, header=None)
panther["name"] = panther.index + " (" + panther[2] + ")"

manual = pd.read_csv("/Users/npapadop/Documents/teaching/2023/advanced_scRNAseq/data/clytia/genes.tsv",
                      sep="\t", index_col=0, header=None)
manual[1] = manual.index + " [" + manual[1] + "]"

var = pd.DataFrame(index=raw.var.index)
var["name"] = var.index
var["name"].replace(manual[1].to_dict(), inplace=True)
var["name"].replace(panther["name"].to_dict(), inplace=True)

In [11]:
for fed in [True, False]:
        for batch in ["0", "1"]:
            keep = (processed.obs["fed"] == fed) & (processed.obs["batch"] == batch)
            name = f"{'fed' if fed else 'starved'}_batch{batch}"
            animal = sc.AnnData(raw.X[keep], obs=processed.obs[keep], var=var)
            animal.write_h5ad(output + "anndata/" + f"{name}.h5ad")